Create Asset Compliance: CURRENT_OUTSTANDING policy
-------------------------------------------------------------------------------------

In [ ]:
# Define a compliance policy that alerts when an asset spends too long in a bad state.

# Main function establishes a connection to RKVST using an App Registration then uses that
# to create an access policy, test it in good and bad states, then cleans up.

In [ ]:
from json import dumps as json_dumps
from os import getenv
from time import sleep
from uuid import uuid4
from warnings import filterwarnings

from archivist.archivist import Archivist
from archivist.compliance_policy_requests import CompliancePolicyCurrentOutstanding
from archivist.constants import ASSET_BEHAVIOURS
from archivist.proof_mechanism import ProofMechanism
from archivist.logger import set_logger

filterwarnings("ignore", message="Unverified HTTPS request")

In [ ]:
# Connection parameters. Fill in with client ID and secret from an appropriately
# permissioned Application Registration
RKVST_URL = "https://app.rkvst.io"
APPREG_CLIENT = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
APPREG_SECRET = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
def create_compliance_policy(arch):
    """Compliance policy which notices when process steps are
    not executed - eg 'you must close the door after you open it'
    or 'candidate software build must be approved before release'

    This example creates a policy that requires doors to be closed
    after they are opened.
    """
    compliance_policy = arch.compliance_policies.create(
        CompliancePolicyCurrentOutstanding(
            description="Vault doors should be closed according to site security policy section Phys.Integ.02",
            display_name="Phys.Integ.02",
            asset_filter=[
                ["attributes.arc_display_type=Vault Door"],
            ],
            event_display_type="Open",
            closing_event_display_type="Close",
        )
    )
    print("CURRENT_OUTSTANDING_POLICY:", json_dumps(compliance_policy, indent=4))
    return compliance_policy

In [ ]:
def create_door(arch):
    """
    Creates an Asset record to track a particular door.
    """

    door, _ = arch.assets.create_if_not_exists(
        {
            "selector": [
                {
                    "attributes": [
                        "arc_display_name",
                        "arc_display_type",
                    ]
                },
            ],
            "behaviours": ASSET_BEHAVIOURS,
            "proof_mechanism": ProofMechanism.SIMPLE_HASH.name,
            "attributes": {
                "arc_display_name": "Gringott's Vault 2",
                "arc_description": "Main door to the second level security vault in Gringott's Wizarding Bank",
                "arc_display_type": "Vault Door",
            },
        },
        confirm=True,
    )
    print("DOOR:", json_dumps(door, indent=4))
    return door

In [ ]:
def open_door(arch, door, tag):
    """
    Open the vault door
    """
    door_opened = arch.events.create(
        door["identity"],
        {
            "operation": "Record",
            "behaviour": "RecordEvidence",
        },
        {
            "arc_description": "Open the door for Lucius Malfoy",
            "arc_display_type": "Open",
            "arc_correlation_value": f"{tag}",
        },
        confirm=True,
    )
    print("DOOR_OPENED:", json_dumps(door_opened, indent=4))

In [ ]:
def close_door(arch, door, tag):
    """
    Close the vault door
    """
    door_closed = arch.events.create(
        door["identity"],
        {
            "operation": "Record",
            "behaviour": "RecordEvidence",
        },
        {
            "arc_description": "Closed the door after Lucius Malfoy exited the vault",
            "arc_display_type": "Close",
            "arc_correlation_value": f"{tag}",
        },
        confirm=True,
    )
    print("DOOR_CLOSED:", json_dumps(door_closed, indent=4))

In [ ]:
"""Main function of compliance sample test.

 * Connect to RKVST with client ID and secret credential
 * Create minimal test objects to demonstrate the feature
 * Test interactions
 * Clean up
"""
# optional call to set the logger level for all subsystems. The argument can
# be either "INFO" or "DEBUG". For more sophisticated logging control see the
# documentation.
set_logger("INFO")

# Initialize connection to RKVST
print("Connecting to RKVST")
arch = Archivist(RKVST_URL, (APPREG_CLIENT, APPREG_SECRET))

In [ ]:
# Compliance policies with related events (eg open/close, order/ship/deliver
# type situations) require events to be linked through a correlation value.
# In many cases this will be obvious (a CVE tag for vulnerability management,
# or a works ticket number for maintenance, or even a timestamp) but here
# we'll just make a UUID to make sure it's unique and this test is repeatable
tag = uuid4()
print(f"Tag for this run: {tag}")

In [ ]:
# make a compliance policy that alerts when doors are left open
compliance_policy = create_compliance_policy(arch)

In [ ]:
# create an asset that matches the assets_filter field in the
# compliance policy.
gringotts_vault = create_door(arch)

In [ ]:
# Open the door
open_door(arch, gringotts_vault, tag)

# Check compliance: should fail because the door is open
sleep(5)
compliance_nok = arch.compliance.compliant_at(
    gringotts_vault["identity"],
)
print("COMPLIANCE (should be false):", json_dumps(compliance_nok, indent=4))

In [ ]:
# Now close the door
close_door(arch, gringotts_vault, tag)

# Check compliance - should be OK because the door is now closed
sleep(5)
compliance_ok = arch.compliance.compliant_at(
    gringotts_vault["identity"],
)
print("COMPLIANCE (should be true):", json_dumps(compliance_ok, indent=4))

In [ ]:
# However the fact that it is OK *now* is a bit of a red herring. It
# was non-compliant in the past and this may be an issue that needs to
# be verified during an investigation, insurance claim, or other dispute.
# We can check the audit history for compliance *at a point in time* and
# get a verifiable answer to the state of that asset at that time.

# To make sure the example works with such short time frames we grab the
# time from the previous not OK compliance call, but you can choose any
# arbitrary time in a real forensic process
time_of_suspicion = compliance_nok["compliant_at"]
compliance_nok = arch.compliance.compliant_at(
    gringotts_vault["identity"], compliant_at=time_of_suspicion
)
print("HISTORICAL COMPLIANCE (should be false):", json_dumps(compliance_nok, indent=4))

In [ ]:
# finally clean up by deleting the compliance_policy
_ = arch.compliance_policies.delete(
    compliance_policy["identity"],
)